In [ ]:
import requests
import csv

In [ ]:
hidden_nav_file = 'hidden_app_courses.csv'
canvas_host = 'canvas.your-institution.edu'
canvas_admin_token = 'your_api_token_goes_here'
external_tool_id = '12345'
headers = {'Authorization': f'Bearer {canvas_admin_token}'}

## 1: use Canvas Data to find hidden Zoom nav items
Run this query against your Canvas Data 2 database(credentials are in the `confluence wiki` and in the `tlt-ops` account) and save to a CSV called `hidden_zoom_courses.csv`. You'll need to replace the numeric IDs with the tool and enrollment term IDs from your instance. Note: we only really need the `id` to make the change. You can find the tool's id in canvas or via the API by making a call to `https://{canvas_host}/api/v1/courses/{course_id}/tabs`. This will list out all available tools and their ID's 

```sql
select id, course_code, created_at, updated_at, sis_source_id, enrollment_term_id, tab_configuration, name
  from canvas.courses as can 
  where can.tab_configuration like '%{"id":"context_external_tool_<tool id>","hidden":true}%' and enrollment_term_id='<term id>';
```


## 2: Unhide the hidden nav items found in step 1

In [ ]:
# unhide navs

with open(hidden_nav_file, 'r') as infile:
    reader = csv.DictReader(infile)
    r = 0;
    for row in reader:
        api_url = f'https://{canvas_host}/api/v1/courses/{row["canvas_id"]}/tabs/context_external_tool_{external_tool_id}?hidden=false'
        response = requests.put(api_url, headers=headers)
        print(response.text)
        r += 1;
        print(f'{r}; ', end='')

        
print('Done!')